In [2]:
import serial
import json
from vpython import box, vector, rate, scene

# Adjust the serial port and baud rate as needed.
SERIAL_PORT = "COM8"  # Change to your actual port (e.g., "COM3" on Windows)
BAUD_RATE = 115200

# Initialize serial connection to the Pico.
ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)

# Create a 3D box representing the sensor.
# Dimensions: 25 mm (x-axis), 10 mm (y-axis), 3 mm (z-axis)
my_box = box(length=0.025, height=0.010, width=0.003, color=vector(1, 0, 0))

def quaternion_to_axis_up(q):
    """
    Convert a quaternion q = [w, x, y, z] into two VPython vectors:
    - axis: represents the rotated x-axis (with x and y components inverted).
    - up: represents the rotated z-axis (with x and y components inverted).
    
    The sensor’s coordinate system may be mounted so that its positive x and y axes
    correspond to negative x and y in the VPython world. By flipping these components,
    a positive tilt along the sensor's x and y becomes a positive rotation in the VPython scene.
    """
    w, x, y, z = q
    
    # Standard conversion: first column of the rotation matrix (rotated x-axis)
    r00 = 1 - 2*(y*y + z*z)
    r10 = 2*(x*y + z*w)
    r20 = 2*(x*z - y*w)
    
    # Standard conversion: third column of the rotation matrix (rotated z-axis)
    r02 = 2*(x*z + y*w)
    r12 = 2*(y*z - x*w)
    r22 = 1 - 2*(x*x + y*y)
    
    # Invert both x and y components to fix the inverted rotation when lifting positive x and y.
    axis = vector(-r00, -r10, r20)
    up   = vector(-r02, -r12, r22)
    
    return axis, up

while True:
    rate(50)  # Limit update rate to about 50 frames per second.
    line = ser.readline().decode().strip()
    if line:
        try:
            data = json.loads(line)
            # Use sensor_1's quaternion for the visualization.
            q = data["sensor_1"]["quat"]
            axis, up = quaternion_to_axis_up(q)
            my_box.axis = axis
            my_box.up = up
        except Exception as e:
            print("Error parsing line:", line, e)


<IPython.core.display.Javascript object>

SerialException: could not open port 'COM8': PermissionError(13, 'Access is denied.', None, 5)